# Hypothesis Testing - Pricing

Author: Dan E

Date: 2023-07-10

Description: This will test different pricing features and the impact on model performance. The following features will be tried and compared to the baseline model.

* Average Price
* Average Super Peak Price
* Price Variance
* Average Super Peak & Price Variance

The testing will be done on a single 80:20 split.

Remark: This is an older version of the model and data than is currently under development.

## Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import platform
from sqlalchemy.engine import URL
from sqlalchemy import create_engine, text
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
def fetch_SQLserver_data(sql_code,conn_string):
	"""
	Uses pyodbc.connect and pandas.read_sql to return query results as dataframe.
	:param sql_code: string, query to be submitted
	:param conn_string: string, full connection details as string
	"""
	connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_string})
	engine = create_engine(connection_url)
	with engine.connect() as conn:
		df = pd.read_sql(sql=text(sql_code), con=conn)
		conn.commit()
	return df

if platform.system() == 'Windows':
	conn_CDM = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-02;DATABASE=CDM;trusted_connection=Yes'
	conn_l0 = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-02;DATABASE=Sykes_DW_Level0_Staging;trusted_connection=Yes'
	conn_l2 = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-02;DATABASE=Sykes_DW_Level2_ADM;trusted_connection=Yes'
	conn_BOX = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-02;DATABASE=Box;trusted_connection=Yes'

In [3]:
def get_query(_file):
	query_file = open(_file,'r')
	query = query_file.read()
	query_file.close()
	return query

## Data

### Pricing

In [4]:
PRICING_SQL = get_query('data/pricing_features_raw_dev.sql')

PRICING_SQL = PRICING_SQL.replace('?startdate', '2022-03-12')
PRICING_SQL = PRICING_SQL.replace('?enddate', '2023-03-11')

pricing_data = fetch_SQLserver_data(PRICING_SQL, conn_CDM)

### Revenue

In [5]:
revenue_data  = pd.read_csv('data/test_v3_nrp_2023_03_16.csv')

### Property Features

In [6]:
SQL_PROPERY_FEATURES = get_query('data/cdm_features_query.sql')

SQL_PROPERY_FEATURES  = SQL_PROPERY_FEATURES.replace('?', '2022-03-12')

property_data = fetch_SQLserver_data(SQL_PROPERY_FEATURES, conn_CDM)

### Owner Occupancy

### Current Model Datasets

In [7]:
train_0 = pd.read_csv('cv-0-train-data-1.csv')
test_0 = pd.read_csv('cv-0-test-data-2.csv')

train_1 = pd.read_csv('cv-1-train-data-1.csv')
test_1 = pd.read_csv('cv-1-test-data-2.csv')

train_2 = pd.read_csv('cv-2-train-data-1.csv')
test_2 = pd.read_csv('cv-2-test-data-2.csv')

train_3 = pd.read_csv('cv-3-train-data-1.csv')
test_3 = pd.read_csv('cv-3-test-data-2.csv')

train_4 = pd.read_csv('cv-4-train-data-1.csv')
test_4 = pd.read_csv('cv-4-test-data-2.csv')

In [8]:
other_0 = pd.read_csv('cv-0-test-data-1.csv')
other_1 = pd.read_csv('cv-1-test-data-1.csv')
other_2 = pd.read_csv('cv-2-test-data-1.csv')
other_3 = pd.read_csv('cv-3-test-data-1.csv')
other_4 = pd.read_csv('cv-4-test-data-1.csv')

## Create New Features

Start with calculating baseline CV model and evaluation of this model here. THe move onto using the new features to start testing the performance increase.

The new features to be tested are as follows:

* Average Price
* Average Super Peak Price
* Flat pricing (low variance pricing)
* Price-Revenue Elasticity (where applicable)

###  Baseline Model

In [15]:
from catboost import CatBoostRegressor
from sklearn.cluster import KMeans

def wape(actual,pred):
    wape = 100*np.abs(actual - pred).sum() / actual.sum()
    return wape

def smape(actual, pred):
    val = 100/len(actual) * np.sum(2 * np.abs(pred - actual) / (np.abs(actual) + np.abs(pred)))
    return val

model_params = {'iterations':400}

initial_cat_features = ['BrandID', 'AllowsPets', 'ChangeOverDay', 'PropertyType',
       'Rating', 'isHoneyPot', 'LocationType', 
       'WorkingRegion', 'RevenueManagementOption', 'hasBroadband', 'hasDishwasher',
       'hasEnclosedGarden', 'hasGames', 'hasGarden', 'hasHotTub',
       'hasIndoorPool', 'hasOffRoadParking', 'hasOpenFire', 'OwnerBookingTier',
       'hasShortBreaksAllowed', 'hasLastMinuteBreakAllowed',
       'OpenFireType', 'daily', 'owner_led_pricing', 'ClusterLabel']

In [17]:
model = CatBoostRegressor(**model_params)

In [11]:
train_data = [train_0, train_1, train_2, train_3, train_4]
test_data = [test_0, test_1, test_2, test_3, test_4]
other_data = [other_0, other_1, other_2, other_3, other_4]

In [18]:
predictions = []
other_predictions = []

for x,y,z in tqdm(zip(train_data, test_data, other_data), total=5):
    X_train = x.drop(['TrainLabel', 'PropertyReference'],axis=1)
    y_train = x['TrainLabel'] 

    X_test = y.drop(['TestLabel', 'PropertyReference'],axis=1)
    X_other = z.drop(['TrainLabel', 'PropertyReference'],axis=1)

    model.fit(
        X=X_train,
        y=y_train,
        cat_features=initial_cat_features,
        plot=False,
        verbose=0
             )

    preds = model.predict(X_test)
    other_preds = model.predict(X_other)

    predictions.append(preds)
    other_predictions.append(other_preds)

  0%|          | 0/5 [00:00<?, ?it/s]

In [19]:
df_evaluate = pd.DataFrame()

df_evaluate['Predictions'] = np.concatenate(predictions)
df_evaluate['Actual'] = np.concatenate([x.TestLabel for x in test_data])

In [20]:
smape(df_evaluate['Actual'], df_evaluate['Predictions'])

24.273552523175276

In [21]:
df_other = pd.DataFrame()

df_other['Predictions'] = np.concatenate(other_predictions)
df_other['Actual'] = np.concatenate([z.TrainLabel for z in other_data])

In [22]:
smape(df_other['Actual'], df_other['Predictions'])

15.155679977333786

### Average Price Feature Inclusion

Require pricing for train and tests set seperately

In [13]:
# train pricing - raw
PRICING_SQL = get_query('data/pricing_features_raw_dev.sql')

PRICING_SQL = PRICING_SQL.replace('?startdate', '2021-03-13')
PRICING_SQL = PRICING_SQL.replace('?enddate', '2022-03-12')

pricing_train = fetch_SQLserver_data(PRICING_SQL, conn_CDM)

In [14]:
# test pricing - raw
PRICING_SQL = get_query('data/pricing_features_raw_dev.sql')

PRICING_SQL = PRICING_SQL.replace('?startdate', '2022-03-12')
PRICING_SQL = PRICING_SQL.replace('?enddate', '2023-03-11')

pricing_test = fetch_SQLserver_data(PRICING_SQL, conn_CDM)

In [23]:
# train aggregated
train_mean_pricing = pricing_train[['PropertyReference', 'BasePriceSterling']].groupby('PropertyReference').mean().reset_index()

In [24]:
# test aggregated
test_mean_pricing = pricing_test[['PropertyReference', 'BasePriceSterling']].groupby('PropertyReference').mean().reset_index()

In [25]:
train_avg = []
test_avg = []
other_avg = []
# join to existing train data
for x,y,z in zip(train_data, test_data, other_data):
    x = x.merge(train_mean_pricing, on='PropertyReference')
    y = y.merge(test_mean_pricing, on='PropertyReference')
    z = z.merge(train_mean_pricing, on='PropertyReference')
    train_avg.append(x)
    test_avg.append(y)
    other_avg.append(z)

In [26]:
predictions = []
other_predictions = []

for x,y,z in tqdm(zip(train_avg, test_avg, other_avg), total=5):
    X_train = x.drop(['TrainLabel', 'PropertyReference'],axis=1)
    y_train = x['TrainLabel'] 

    X_test = y.drop(['TestLabel', 'PropertyReference'],axis=1)
    X_other = z.drop(['TrainLabel', 'PropertyReference'],axis=1)

    model.fit(
        X=X_train,
        y=y_train,
        cat_features=initial_cat_features,
        plot=False,
        verbose=0
             )

    preds = model.predict(X_test)
    other_preds = model.predict(X_other)

    predictions.append(preds)
    other_predictions.append(other_preds)

  0%|          | 0/5 [00:00<?, ?it/s]

In [27]:
df_evaluate_avg = pd.DataFrame()

df_evaluate_avg['Predictions'] = np.concatenate(predictions)
df_evaluate_avg['Actual'] = np.concatenate([x.TestLabel for x in test_avg])

In [28]:
# slight drop - but lots of features and pricing
smape(df_evaluate_avg['Actual'], df_evaluate_avg['Predictions'])

23.744346136528502

In [29]:
len(other_predictions)

5

In [30]:
df_other = pd.DataFrame()

df_other['Predictions'] = np.concatenate(other_predictions)
df_other['Actual'] = np.concatenate([z.TrainLabel for z in other_data])

In [31]:
# big improvement here
smape(df_other['Actual'], df_other['Predictions'])

12.906000779454708

### Avg Super Peak Price Feature Inclusion

In [32]:
train_mean_pricing = pricing_train.loc[pricing_train['HolidaySeason'] == 'Super Peak'][['PropertyReference', 'BasePriceSterling']].groupby('PropertyReference').mean().reset_index()

In [33]:
test_mean_pricing = pricing_test.loc[pricing_test['HolidaySeason'] == 'Super Peak'][['PropertyReference', 'BasePriceSterling']].groupby('PropertyReference').mean().reset_index()

In [34]:
train_avg = []
test_avg = []
other_avg = []
# join to existing train data
for x,y,z in zip(train_data, test_data, other_data):
    x = x.merge(train_mean_pricing, on='PropertyReference')
    y = y.merge(test_mean_pricing, on='PropertyReference')
    z = z.merge(train_mean_pricing, on='PropertyReference')
    train_avg.append(x)
    test_avg.append(y)
    other_avg.append(z)

In [35]:
predictions = []
other_predictions = []

for x,y,z in tqdm(zip(train_avg, test_avg, other_avg), total=5):
    X_train = x.drop(['TrainLabel', 'PropertyReference'],axis=1)
    y_train = x['TrainLabel'] 

    X_test = y.drop(['TestLabel', 'PropertyReference'],axis=1)
    X_other = z.drop(['TrainLabel', 'PropertyReference'],axis=1)

    model.fit(
        X=X_train,
        y=y_train,
        cat_features=initial_cat_features,
        plot=False,
        verbose=0
             )

    preds = model.predict(X_test)
    other_preds = model.predict(X_other)

    predictions.append(preds)
    other_predictions.append(other_preds)

  0%|          | 0/5 [00:00<?, ?it/s]

In [36]:
df_evaluate_avg = pd.DataFrame()

df_evaluate_avg['Predictions'] = np.concatenate(predictions)
df_evaluate_avg['Actual'] = np.concatenate([x.TestLabel for x in test_avg])

In [37]:
# worse, over-predictions also
smape(df_evaluate_avg['Actual'], df_evaluate_avg['Predictions'])

24.78802437655852

In [38]:
smape(df_evaluate_avg['Actual'], df_evaluate_avg['Predictions']*0.90)

22.952873395114974

In [39]:
df_other = pd.DataFrame()

df_other['Predictions'] = np.concatenate(other_predictions)
df_other['Actual'] = np.concatenate([z.TrainLabel for z in other_data])

In [40]:
smape(df_other['Actual'], df_other['Predictions'])

12.721297258996966

### Flat/Low Variance Pricing

In [41]:
train_mean_pricing = pricing_train[['PropertyReference', 'BasePriceSterling']].groupby('PropertyReference').std().reset_index()

In [42]:
test_mean_pricing = pricing_test[['PropertyReference', 'BasePriceSterling']].groupby('PropertyReference').std().reset_index()

In [43]:
train_avg = []
test_avg = []
other_avg = []
# join to existing train data
for x,y,z in zip(train_data, test_data, other_data):
    x = x.merge(train_mean_pricing, on='PropertyReference')
    y = y.merge(test_mean_pricing, on='PropertyReference')
    z = z.merge(train_mean_pricing, on='PropertyReference')
    train_avg.append(x)
    test_avg.append(y)
    other_avg.append(z)

In [44]:
predictions = []
other_predictions = []

for x,y,z in tqdm(zip(train_avg, test_avg, other_avg), total=5):
    X_train = x.drop(['TrainLabel', 'PropertyReference'],axis=1)
    y_train = x['TrainLabel'] 

    X_test = y.drop(['TestLabel', 'PropertyReference'],axis=1)
    X_other = z.drop(['TrainLabel', 'PropertyReference'],axis=1)

    model.fit(
        X=X_train,
        y=y_train,
        cat_features=initial_cat_features,
        plot=False,
        verbose=0
             )

    preds = model.predict(X_test)
    other_preds = model.predict(X_other)

    predictions.append(preds)
    other_predictions.append(other_preds)

  0%|          | 0/5 [00:00<?, ?it/s]

In [45]:
df_evaluate_avg = pd.DataFrame()

df_evaluate_avg['Predictions'] = np.concatenate(predictions)
df_evaluate_avg['Actual'] = np.concatenate([x.TestLabel for x in test_avg])

In [46]:
smape(df_evaluate_avg['Actual'], df_evaluate_avg['Predictions'])

24.322249864582645

In [47]:
df_other = pd.DataFrame()

df_other['Predictions'] = np.concatenate(other_predictions)
df_other['Actual'] = np.concatenate([z.TrainLabel for z in other_data])

In [48]:
smape(df_other['Actual'], df_other['Predictions'])

13.314344589369096

### Super Peak Average and Variance Pricing

In [49]:
train_mean_pricing_1 = pricing_train[['PropertyReference', 'BasePriceSterling']].groupby('PropertyReference').mean().reset_index()
train_mean_pricing_2 = pricing_train[['PropertyReference', 'BasePriceSterling']].groupby('PropertyReference').std().reset_index()

In [50]:
test_mean_pricing_1 = pricing_test[['PropertyReference', 'BasePriceSterling']].groupby('PropertyReference').mean().reset_index()
test_mean_pricing_2 = pricing_test[['PropertyReference', 'BasePriceSterling']].groupby('PropertyReference').std().reset_index()

In [51]:
train_avg = []
test_avg = []
other_avg = []
# join to existing train data
for x,y,z in zip(train_data, test_data, other_data):
    x = x.merge(train_mean_pricing_1, on='PropertyReference')
    y = y.merge(test_mean_pricing_1, on='PropertyReference')
    z = z.merge(train_mean_pricing_1, on='PropertyReference')
    x = x.merge(train_mean_pricing_2, on='PropertyReference')
    y = y.merge(test_mean_pricing_2, on='PropertyReference')
    z = z.merge(train_mean_pricing_2, on='PropertyReference')
    train_avg.append(x)
    test_avg.append(y)
    other_avg.append(z)

In [52]:
predictions = []
other_predictions = []

for x,y,z in tqdm(zip(train_avg, test_avg, other_avg), total=5):
    X_train = x.drop(['TrainLabel', 'PropertyReference'],axis=1)
    y_train = x['TrainLabel'] 

    X_test = y.drop(['TestLabel', 'PropertyReference'],axis=1)
    X_other = z.drop(['TrainLabel', 'PropertyReference'],axis=1)

    model.fit(
        X=X_train,
        y=y_train,
        cat_features=initial_cat_features,
        plot=False,
        verbose=0
             )

    preds = model.predict(X_test)
    other_preds = model.predict(X_other)

    predictions.append(preds)
    other_predictions.append(other_preds)

  0%|          | 0/5 [00:00<?, ?it/s]

In [53]:
df_evaluate_avg = pd.DataFrame()

df_evaluate_avg['Predictions'] = np.concatenate(predictions)
df_evaluate_avg['Actual'] = np.concatenate([x.TestLabel for x in test_avg])

In [54]:
smape(df_evaluate_avg['Actual'], df_evaluate_avg['Predictions'])

24.004299189056795

In [55]:
df_other = pd.DataFrame()

df_other['Predictions'] = np.concatenate(other_predictions)
df_other['Actual'] = np.concatenate([z.TrainLabel for z in other_data])

In [56]:
# very little than using the super peak average on its own
smape(df_other['Actual'], df_other['Predictions'])

12.683557090969446

## Summary

|Model|Year 1 sMAPE|Year 2 sMAPE|
|-----|------------|------------|
|Baseline|    15.16|       24.27|
|Average|     12.91|       23.74|
|Super Peak| 12.72 |       24.79|
|Variance|    13.31|       24.32|
|Super Peak & Variance|12.68|24.00|

Note: The pricing feature improves historic revenue prediction more than the forecast revenue prediction. It still improves the forecast prediction but not as significantly as it should. This could be due to changing global trends. Including price will also impact the counter-factual evaluation.